<a href="https://colab.research.google.com/github/Rameshwar1812/Detected-Languages/blob/main/Detected_Languages.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install langdetect


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 50.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=321850c14d43d24e3e9108c1602d6f7407c146479ae8996e664c609923c2a4a5
  Stored in directory: /root/.cache/pip/wheels/0a/f2/b2/e5ca405801e05eb7c8ed5b3b4bcf1fcabcd6272c167640072e
Successfully built langdetect


In [ ]:
pip install langid


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 33.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langid: filename=langid-1.1.6-py3-none-any.whl size=1941171 sha256=a8e093e51682696fb34e02664852dbfe27d88124daa3af267e7b6ad0b5c7852d
  Stored in directory: /root/.cache/pip/wheels/32/6a/b6/b7eb43a6ad55b139c15c5daa29f3707659cfa6944d3c696f5b
Successfully built langid


In [ ]:
pip install nltk


In [ ]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import logging
logging.basicConfig(level=logging.ERROR)


In [ ]:
!pip install hmmlearn==0.2.5


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.2/149.2 kB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for hmmlearn
  Running setup.py clean for hmmlearn
Failed to build hmmlearn
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (hmmlearn)


In [ ]:
import warnings
warnings.filterwarnings("ignore", message="The default value of `n_init` will change from 10 to 'auto' in 1.4")


In [ ]:

import sys
import warnings
import nltk
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from hmmlearn import hmm

# Suppress warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
    warnings.filterwarnings("ignore", category=DeprecationWarning)
    warnings.filterwarnings("ignore", category=FutureWarning)
    warnings.filterwarnings("ignore", category=UserWarning)


languages = pd.read_csv('/content/Language Detection.csv')

# Split the data into training and testing sets
train_data, test_data = train_test_split(languages, test_size=0.2)

# Function to extract N-gram features
def extract_ngrams(text, n):
    tokens = word_tokenize(text)
    return [' '.join(gram) for gram in ngrams(tokens, n)]

# Extract N-gram features for the training data
X_train = train_data['Text']
y_train = train_data['Language']

# Use CountVectorizer to convert text data into a matrix of token counts
vectorizer = CountVectorizer()
X_train_count = vectorizer.fit_transform(X_train)

# Train the Naive Bayes classifier
nb_classifier = MultinomialNB()
nb_classifier.fit(X_train_count, y_train)

# Train the Decision Tree classifier
dt_classifier = DecisionTreeClassifier()
dt_classifier.fit(X_train_count, y_train)

# Train the HMM model for each language
hmm_models = {}
for language in languages['Language'].unique():
    language_data = languages[languages['Language'] == language]
    hmm_model = hmm.GaussianHMM(n_components=2)  # Use GaussianHMM for faster training
    hmm_model.fit(vectorizer.transform(language_data['Text']).toarray())
    hmm_models[language] = hmm_model

# Function to predict languages using HMM and Naive Bayes
def predict_languages(text):
    text_count = vectorizer.transform([text])
    nb_prediction = nb_classifier.predict(text_count)
    hmm_predictions = {}
    for language, hmm_model in hmm_models.items():
        hmm_predictions[language] = hmm_model.score(text_count.toarray())
    return nb_prediction, hmm_predictions

# Evaluate the model on the testing data
test_texts = test_data['Text']
test_languages = test_data['Language']
predicted_languages = []
for text in test_texts:
    nb_prediction, hmm_predictions = predict_languages(text)
    predicted_language = max(hmm_predictions, key=hmm_predictions.get)
    predicted_languages.append(predicted_language)

accuracy = accuracy_score(test_languages, predicted_languages)
print('Accuracy:', accuracy)



ModuleNotFoundError: No module named 'hmmlearn'

# New Section

# New Section

In [ ]:
def predict_sentence_languages(text):
    words = word_tokenize(text)
    detected_languages = set()

    for word in words:
        word_count = vectorizer.transform([word])
        nb_prediction = nb_classifier.predict(word_count)[0]
        hmm_predictions = {}

        # Get HMM predictions for the word
        for language, hmm_model in hmm_models.items():
            hmm_predictions[language] = hmm_model.score(word_count.toarray())

        # Get the HMM prediction with the highest score
        hmm_prediction = max(hmm_predictions, key=hmm_predictions.get)

        # Add both Naive Bayes and HMM predictions to the detected languages set
        detected_languages.add(nb_prediction)
        detected_languages.add(hmm_prediction)

    # Return the unique languages detected in the sentence
    return list(detected_languages)

# Example input text
text = input()
predicted_languages = predict_sentence_languages(text)

# Print the detected languages in the sentence
print("Detected Languages:", predicted_languages)


In [ ]:
def predict_sentence_languages(text, hmm_threshold=-1000):  # Set a reasonable score threshold
    words = word_tokenize(text)
    detected_languages = set()

    for word in words:
        word_count = vectorizer.transform([word])
        nb_prediction = nb_classifier.predict(word_count)[0]
        hmm_predictions = {}

        # Get HMM predictions for the word
        for language, hmm_model in hmm_models.items():
            score = hmm_model.score(word_count.toarray())
            if score > hmm_threshold:  # Only consider languages with scores above the threshold
                hmm_predictions[language] = score

        # Add the Naive Bayes prediction
        detected_languages.add(nb_prediction)

        # Add the highest scoring HMM prediction if any
        if hmm_predictions:
            hmm_prediction = max(hmm_predictions, key=hmm_predictions.get)
            detected_languages.add(hmm_prediction)

    # Return the unique languages detected in the sentence
    return list(detected_languages)

# Example input text
text = input()
predicted_languages = predict_sentence_languages(text)

# Print the detected languages in the sentence
print("Detected Languages:", predicted_languages)


Let’s go out for dinner, but I want to eat some 寿司 (sushi)Let’s go out for dinner, but I want to eat some 寿司 (sushi)Let’s go out for dinner, but I want to eat some 寿司 (sushi)Let’s go out for dinner, but I want to eat some 寿司 (sushi)Let’s go out for dinner, but I want to eat some 寿司 (sushi)


LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - '/root/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************
